In [1]:
import numpy as np

In [15]:
matrix = np.random.randint(1, 100000, size=(4, 9))
matrix_sorted = np.sort(matrix, axis=1)
print(matrix_sorted)

zeros = np.zeros((4, 1), dtype=int)
lakhs = np.full((4, 1), 100000, dtype=int)
breaks = np.hstack((zeros, matrix_sorted, lakhs))

print(breaks)

[[11613 20607 38091 38960 51601 54980 65450 65471 70582]
 [ 3883 18199 33932 39420 40325 51054 53175 92345 94959]
 [13508 26448 31532 42975 45000 67163 70749 84999 99626]
 [ 6830 14283 14599 18313 30196 40809 43599 47134 79408]]
[[     0  11613  20607  38091  38960  51601  54980  65450  65471  70582
  100000]
 [     0   3883  18199  33932  39420  40325  51054  53175  92345  94959
  100000]
 [     0  13508  26448  31532  42975  45000  67163  70749  84999  99626
  100000]
 [     0   6830  14283  14599  18313  30196  40809  43599  47134  79408
  100000]]


In [16]:
matrix_2 = np.random.rand(4, 10)
slopes = -np.sort(-matrix_2, axis=1)
print(slopes)

[[0.85209025 0.78134701 0.62679866 0.5414062  0.49284386 0.42882488
  0.38166284 0.37649195 0.31517853 0.27717003]
 [0.97101573 0.80198713 0.73545965 0.65895804 0.65635218 0.47545407
  0.47008674 0.2657316  0.18130755 0.11140761]
 [0.66281525 0.45946924 0.36509719 0.34922865 0.31011256 0.27878471
  0.23399165 0.19507865 0.04722654 0.01824707]
 [0.94234152 0.81296938 0.77814891 0.69450167 0.57985917 0.51838066
  0.42917744 0.16653651 0.10625614 0.09189275]]


In [17]:
from gurobipy import Model, GRB

In [19]:
prob_3 = Model("Ad Budget Allocation")

Restricted license - for non-production use only - expires 2026-11-23


In [23]:
x = {}
for i in range(breaks.shape[0]):
    for s in range(breaks.shape[1] - 1):
        upper_bound = breaks[i, s+1] - breaks[i, s]
        x[i, s] = prob_3.addVar(lb=0, ub=upper_bound)

y = {}
for i in range(breaks.shape[0]):
    y[i] = prob_3.addVar(vtype=GRB.BINARY)

In [24]:
prob_3.setObjective(
    sum(slopes[i, s] * x[i, s] for i in range(breaks.shape[0]) for s in range(breaks.shape[1] - 1)),
    GRB.MAXIMIZE
)

In [25]:
prob_3.addConstr(
    sum(x[i, s] for i in range(breaks.shape[0]) for s in range(breaks.shape[1] - 1)) <= 100000,
    "TotalBudget"
)

for i in range(breaks.shape[0]):
    prob_3.addConstr(
        sum(x[i, s] for s in range(breaks.shape[1] - 1)) <= 100000 * y[i]
    )

prob_3.addConstr(
    sum(y[i] for i in range(breaks.shape[0])) <= 3,
    name="max_3_active_channels"
)

tv_index = 1 #Tv is second channel
prob_3.addConstr(
    sum(x[tv_index, s] for s in range(breaks.shape[1] - 1)) >= 0.3 * 100000 * y[tv_index],
    name="tv_minimum_if_used"
)

<gurobi.Constr *Awaiting Model Update*>

In [32]:
prob_3.setParam("OutputFlag", False)
prob_3.optimize()

In [31]:
print("Objective Function Value: %f" % prob_3.objVal)

Objective Function Value: 76132.002831


In [33]:
x_matrix = np.zeros((breaks.shape[0], breaks.shape[1] - 1))

for i in range(breaks.shape[0]):
    for s in range(breaks.shape[1] - 1):
        x_matrix[i, s] = x[i, s].X  

np.set_printoptions()
print("\nOptimal Budget Allocation Matrix (rows=channels, cols=segments):")
print(x_matrix)


Optimal Budget Allocation Matrix (rows=channels, cols=segments):
[[11613.  8994. 17484.     0.     0.     0.     0.     0.     0.     0.]
 [ 3883. 14316. 15733.  5488.   905.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [ 6830.  7453.   316.  3714.  3271.     0.     0.     0.     0.     0.]]
